In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib 
import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc
from sklearn import metrics
from sklearn.decomposition import PCA
import tabulate
import itertools
import igraph as ig
import matplotlib.patches as mpatches
from scipy.ndimage import gaussian_filter1d
from sklearn.neighbors import kneighbors_graph
from scipy.spatial.distance import pdist, squareform
import leidenalg as la
import dask.array as da
from sklearn.cluster import SpectralClustering
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
import umap
import bbknn
import bbknn.matrix
from importlib import reload
from scipy import stats
import matplotlib.image as mpimg
import goatools
from goatools.anno.gaf_reader import GafReader
import Bio.UniProt.GOA as GOA
import gget

# locals
import utils as ut
reload(ut)

2022-12-12 14:51:26.344290: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-12 14:51:26.459667: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/slurm/lib64:
2022-12-12 14:51:26.459689: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-12 14:51:26.485094: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-12 14:51:27.1682

<module 'utils' from '/home/cstansbu/git_repositories/spatial_transcriptomics/clustering/utils.py'>

In [2]:
# load cluster assignments
cdf = pd.read_csv("clusterAssignments.csv")
cid = {}

for key in cdf['key'].unique():
    t = cdf[cdf['key'] == key]
    cid[key] = t
    
print('done')

done


In [3]:
dirPath = "/nfs/turbo/umms-indikar/shared/projects/spatial_transcriptomics/data/scanpy/"

adata = {}

for f in os.listdir(dirPath):
    
    if f.endswith('.h5ad'):
        key = f.split(".")[0]
        fullPath = f"{dirPath}{f}"
        data = sc.read(fullPath)
        adata[key] = data
    
print("done")

done


In [21]:
keys = ['ND', 'HFD8', 'HFD14']
outpath = "/nfs/turbo/umms-indikar/shared/projects/spatial_transcriptomics/data/cellAssignments/"

res = []
refs = []

for key in keys:
    data = adata[key]
    
    # get the run key
    allKeys = list(data.obs.keys() )
    allKeys.remove('n_genes')
    runKey = allKeys[0]
    
    clusterIds = data.obs[runKey].astype(int) + 1  # note that these are zero-indexed!
    clusterIds = pd.DataFrame(clusterIds).reset_index(drop=False)
    clusterIds.columns =['barcode', 'clusterId']
    
    df = data.to_df()
    
    ctf = pd.DataFrame(df.index)
    ctf.columns =['barcode']
    ctf = pd.merge(ctf, clusterIds,
                   how='left',
                   left_on='barcode', 
                   right_on='barcode')
    
    cTypes = cid[key][['cellType', 'clusterId']]
    
    ctf = pd.merge(ctf, cTypes,
                   how='left',
                   left_on='clusterId', 
                   right_on='clusterId')
    
    fpath = f"{outpath}{key}.csv"
    ctf.to_csv(fpath, index=False)
    print(f'done: {fpath}')


done: /nfs/turbo/umms-indikar/shared/projects/spatial_transcriptomics/data/cellAssignments/ND.csv
done: /nfs/turbo/umms-indikar/shared/projects/spatial_transcriptomics/data/cellAssignments/HFD8.csv
done: /nfs/turbo/umms-indikar/shared/projects/spatial_transcriptomics/data/cellAssignments/HFD14.csv
